In [2]:
import pandas as pd
import requests
import json
import pickle

# Load the CSV file
df = pd.read_csv('FIT1043-MusicGenre-Dataset.csv')

# Spotify API URL
SPOTIFY_API_URL = 'https://api.spotify.com/v1/'

# Spotify API token
# You need to get this from your Spotify Developer Dashboard
SPOTIFY_API_TOKEN = 'BQDhbAg3itGd4ukhUyNJpRmKr8zNodl1gR9tkdjYNoYr-1VcBZU8MkKQydJgGsIMALMBv7bFr12dJI1gYesyFTunwzxIr73LXXh87-rk_y7chpjM0t8'

# Function to get track details
def get_track_details(track_name):
    response = requests.get(
        SPOTIFY_API_URL + 'search',
        headers={
            'Authorization': 'Bearer ' + SPOTIFY_API_TOKEN
        },
        params={
            'q': track_name,
            'type': 'track',
            'limit': 1
        }
    )
    track_data = response.json()
    artist_id = track_data['tracks']['items'][0]['artists'][0]['id']
    
    # Get artist details
    artist_response = requests.get(
        SPOTIFY_API_URL + 'artists/' + artist_id,
        headers={
            'Authorization': 'Bearer ' + SPOTIFY_API_TOKEN
        }
    )
    artist_data = artist_response.json()
    genre = artist_data['genres'][0] if artist_data['genres'] else 'Unknown genre'
    
    return track_data, genre

# Create an empty dictionary to store the mapping
genre_mapping = {}

# For each track name, get the track details and extract the genre
for index, row in df.iterrows():
    track_name = row['track_name']
    existing_genre = row['music_genre']
    track_data, spotify_genre = get_track_details(track_name)
    
    # Add the mapping to the dictionary
    genre_mapping[spotify_genre] = existing_genre
    if index % 50 == 0:
        print('Processed', index, 'tracks')

with open('genre_mapping.pkl', 'wb') as f:
    pickle.dump(genre_mapping, f)

Processed 0 tracks
Processed 50 tracks
Processed 100 tracks
Processed 150 tracks
Processed 200 tracks
Processed 250 tracks
Processed 300 tracks
Processed 350 tracks
Processed 400 tracks
Processed 450 tracks
Processed 500 tracks
Processed 550 tracks
Processed 600 tracks
Processed 650 tracks
Processed 700 tracks
Processed 750 tracks
Processed 800 tracks
Processed 850 tracks
Processed 900 tracks
Processed 950 tracks
Processed 1000 tracks
Processed 1050 tracks
Processed 1100 tracks
Processed 1150 tracks
Processed 1200 tracks
Processed 1250 tracks
Processed 1300 tracks
Processed 1350 tracks
Processed 1400 tracks


IndexError: list index out of range

In [ ]:
# Print the genre mapping
for spotify_genre, existing_genre in genre_mapping.items():
    print(f'Spotify Genre: {spotify_genre}, Existing Genre: {existing_genre}')

In [ ]:
# Load the genre mapping from the pickle file
with open('genre_mapping.pkl', 'rb') as f:
    genre_mapping = pickle.load(f)

# If not found in genre_mapping, set to default:
genre_keywords = {
    'alternative pop': 0,
    'japanese anime': 1,
    'blues': 2,
    'classical': 3,
    'country folk': 4,
    'electronic': 5,
    'hip hop': 6,
    'jazz': 7,
    'rap': 8,
    'rock metal': 9
}

def get_genre_code(genre):
    try:
        return genre_mapping[genre]
    except KeyError:
        for keyword, code in genre_keywords.items():
            if keyword in genre:
                return code
        return 0

df = pd.read_csv('FIT1043-MusicGenre-Submission.csv')
# For each track name, get the track details and extract the genre
for index, row in df.iterrows():
    track_name = row['track_name']
    track_data, genre = get_track_details(track_name)
    genre_code = get_genre_code(genre)
    df.at[index, 'music_genre'] = genre_code
    if index % 50 == 0:
        print('Processed', index, 'tracks')

df.to_csv('FIT1043-Training.csv', index=False)